In [1]:
import pandas as pd
import sqlite3 
import requests
from bs4 import BeautifulSoup 

In [2]:
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'

db_name = 'Movies.db'

In [3]:
# Getting the html
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

# Getting the tables from html
tables = data.find_all('tbody')
n_tables = len(tables)
print(f'Number of tables: {n_tables}')

Number of tables: 3


# Extraction

In [48]:
extracted_tables = []

for i in range(n_tables):

  # Getting the columns names
  header = tables[i].find_all('th')
  header_cols = [col.contents[0] for col in header]
  n_cols = len(header_cols)

  # Getting the other rows
  rows = tables[i].find_all('tr')

  extracted_data = pd.DataFrame(columns=header_cols)

  # First row is header
  for row in rows[1:-1]:
    cols = row.find_all('td')

    # Create a list from the row values
    _values = [value.contents[0] for value in cols]
    # Convert the list into a dictionary
    _dict = {key: _values[i] for i,key in enumerate(header_cols)}
    # Convert the dictionary into a dataframe
    _df = pd.DataFrame(_dict, index=[0])
    extracted_data = pd.concat([extracted_data,_df], ignore_index=True)

  extracted_tables.append(extracted_data)

In [32]:
extracted_tables[0]

,Average Rank,Film,Year,Rotten Tomatoes' Top 100,IMDb's Top 250,Empire's Top 100,AFI's Top 100,BFI's Top 100
0,1,The Godfather,1972,17,2,3,2,21\n
1,2,Citizen Kane,1941,2,93,40,1,2\n
2,3,Casablanca,1942,8,42,33,3,84\n
3,4,"The Godfather, Part II",1974,99,4,14,32,31\n
4,5,Singin' in the Rain,1952,52,85,67,5,20\n
...,...,...,...,...,...,...,...,...
102,103,LA Confidential,1997,90,unranked,80,unranked,unranked\n
103,104,Titanic,1997,unranked,unranked,88,83,unranked\n
104,105,Toy Story,1995,unranked,75,unranked,99,unranked\n
105,106,Reservoir Dogs,1992,unranked,90,100,unranked,unranked\n


# Transforming

In [54]:
# We must remove the line breaker symbol in the last columns
for i,table in enumerate(extracted_tables):
  # Delete '\n' from table columns
  table.rename(columns={table.columns[-1] : table.columns[-1].strip()}, inplace=True)  

  _cols = table.columns
  # Get the last column from the dataframe
  _values = table[_cols[-1]].values
  for i in range(len(_values)):
    _values[i] = _values[i].strip()
  
extracted_tables[0]

,Average Rank,Film,Year,Rotten Tomatoes' Top 100,IMDb's Top 250,Empire's Top 100,AFI's Top 100,BFI's Top 100
0,1,The Godfather,1972,17,2,3,2,21
1,2,Citizen Kane,1941,2,93,40,1,2
2,3,Casablanca,1942,8,42,33,3,84
3,4,"The Godfather, Part II",1974,99,4,14,32,31
4,5,Singin' in the Rain,1952,52,85,67,5,20
...,...,...,...,...,...,...,...,...
102,103,LA Confidential,1997,90,unranked,80,unranked,unranked
103,104,Titanic,1997,unranked,unranked,88,83,unranked
104,105,Toy Story,1995,unranked,75,unranked,99,unranked
105,106,Reservoir Dogs,1992,unranked,90,100,unranked,unranked


# Storing 

In [58]:
# Creating .csv files for the tables
names = ['TopMovies', 'Decades', 'Genres']

# Creating a SQLite3 database
conn = sqlite3.connect('./data/' + db_name)

for i,table in enumerate(extracted_tables):
  table.to_csv('./data/' + names[i] + '.csv', index=False)
  # Save the table to the database
  table.to_sql(names[i], conn, if_exists='replace', index=False)

conn.close()

# Querying

In [68]:
conn = sqlite3.connect('./data/' + db_name)

query_statement = f"SELECT * FROM {names[0]}" # Query the whole table
query_output = pd.read_sql(query_statement, conn)
print(query_statement)
print(query_output)

query_statement = f"SELECT Film FROM {names[0]}" # Query a specific column 
query_output = pd.read_sql(query_statement, conn)
print(query_statement)
print(query_output)

conn.close()

SELECT * FROM TopMovies
    Average Rank                    Film  Year Rotten Tomatoes' Top 100  \
0              1           The Godfather  1972                       17   
1              2            Citizen Kane  1941                        2   
2              3              Casablanca  1942                        8   
3              4  The Godfather, Part II  1974                       99   
4              5     Singin' in the Rain  1952                       52   
..           ...                     ...   ...                      ...   
102          103         LA Confidential  1997                       90   
103          104                 Titanic  1997                 unranked   
104          105               Toy Story  1995                 unranked   
105          106          Reservoir Dogs  1992                 unranked   
106          107            Paddington 2  2018                       93   

    IMDb's Top 250  Empire's Top 100  AFI's Top 100  BFI's Top 100  
0     